In [1]:
from audio_vessel_classifier import config

In [1]:
from audio_vessel_classifier import config
CONF = config.CONF

# Check and save the configuration
config.check_conf(conf=CONF)
config.conf_dict = config.get_conf_dict(conf=CONF)

In [14]:
import torch
import io

# Path to your .pt file
pt_path = "/srv/DEEP-OC-underwater-noise-classification/data/G_15810_2022-02-07_08-21-29_156-760693_Cargo_underway-using-engine_12-3_1768 (1).pt"

# Read the file content as bytes (simulating upload)
with open(pt_path, "rb") as f:
    file_bytes = f.read()

# Wrap in a BytesIO object to simulate file upload
file_like = io.BytesIO(file_bytes)

# Load the tensor
embedding = torch.load(file_like, map_location="cpu")

# Show basic info
print(type(embedding))
print(getattr(embedding, 'shape', 'No shape (not a tensor?)'))


<class 'torch.Tensor'>
torch.Size([1, 1, 1001, 64])


In [4]:
from audio_vessel_classifier import config
from audio_vessel_classifier.misc import _catch_error
from transformers import AutoProcessor, ClapModel, ClapAudioModelWithProjection, ClapProcessor
from torch import nn

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
clap_model = ClapAudioModelWithProjection.from_pretrained(
    "/srv/DEEP-OC-underwater-noise-classification/models/fine_tuning/model"
).to(device)
clap_model.eval()

# Load the linear classification head
linear_model = nn.Linear(in_features=512, out_features=11)
linear_model.load_state_dict(torch.load(
    "/srv/DEEP-OC-underwater-noise-classification/models/fine_tuning/model/linear.pth",
    map_location=device
))
linear_model = linear_model.to(device)
linear_model.eval()

Linear(in_features=512, out_features=11, bias=True)

In [10]:
def return_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"Selected CUDA device: {torch.cuda.get_device_name(device)}")
    else:
        print("CUDA is not available. Using CPU.")
        device = torch.device('cpu')
    return device
device=return_device()

CUDA is not available. Using CPU.


In [12]:
import torch
import io

# Path to your .pt file
pt_path = "/srv/DEEP-OC-underwater-noise-classification/data/G_15810_2022-02-07_08-21-29_156-760693_Cargo_underway-using-engine_12-3_1768 (1).pt"

# Read the file content as bytes (simulating upload)
with open(pt_path, "rb") as f:
    file_bytes = f.read()

# Wrap in a BytesIO object to simulate file upload
file_like = io.BytesIO(file_bytes)

# Load the tensor
embedding = torch.load(file_like, map_location="cpu")

# Show basic info
print(type(embedding))
print(getattr(embedding, 'shape', 'No shape (not a tensor?)'))

raw_input=embedding
if raw_input.dim() == 2:  # assuming (features, time)
    raw_input = raw_input.unsqueeze(0)

with torch.no_grad():
    clap_output = clap_model(raw_input.to(device))

    # Pass through linear classification head
    output = linear_model(clap_output.audio_embeds)
    predicted_class = torch.argmax(output, dim=1).item()
    probabilities = torch.softmax(output, dim=1).squeeze().cpu().tolist()


<class 'torch.Tensor'>
torch.Size([1, 1, 1001, 64])


NameError: name 'clap_model' is not defined

In [6]:
import torch
import torch.nn as nn

class Fine_tuning_CLAPModel(nn.Module):
    def __init__(self, clap_model, linear_model):
        super().__init__()
        self.clap_model = clap_model
        self.linear_model = linear_model

    def forward(self, x):
        if x.dim() == 2:  # (features, time)
            x = x.unsqueeze(0)

        with torch.no_grad():
            output = self.clap_model(x.to(x.device))

        embeddings = output.audio_embeds
        logits = self.linear_model(embeddings)
        return logits


class Feature_extraction_CLAPModel(nn.Module):
    def __init__(self, fc1,fc2,relu):
        super().__init__()
        self.fc1 = fc1
        self.fc2 = fc2
        self.relu=relu

    def forward(self, x):
        if x.dim() == 2:  # (features, time)
            x = x.unsqueeze(0)

        with torch.no_grad():
            out = self.fc1(x)
            out = self.relu(out)
            out = self.fc2(out)
        return out



def model_loader(device, freeze=True):
    if freeze:
        fc1 = nn.Linear(in_features=512, out_features=256)
        fc2 = nn.Linear(in_features=256, out_features=11)
        relu = nn.ReLU()
    
    
        fc1.load_state_dict(torch.load(
            "/srv/DEEP-OC-underwater-noise-classification/models/feature_extraction/model/fc1.pth",
            map_location=device
        ))
    
        fc2.load_state_dict(torch.load(
            "/srv/DEEP-OC-underwater-noise-classification/models/feature_extraction/model/fc2.pth",
            map_location=device
        ))
    
        relu = nn.ReLU()
        model = Feature_extraction_CLAPModel(fc1, fc2, relu).to(device)
        model.eval()
        return model
    else:    
        # Load CLAP model
        clap_model = ClapAudioModelWithProjection.from_pretrained(
            "/srv/DEEP-OC-underwater-noise-classification/models/fine_tuning/model"
        ).to(device)
        clap_model.eval()
    
        # Load linear head
        linear_model = nn.Linear(in_features=512, out_features=11)
        linear_model.load_state_dict(torch.load(
            "/srv/DEEP-OC-underwater-noise-classification/models/fine_tuning/model/linear.pth",
            map_location=device
        ))
        linear_model = linear_model.to(device)
        linear_model.eval()
        model=Fine_tuning_CLAPModel(clap_model, linear_model).to(device)
        model.eval()
        return model


class Feature_extraction_CLAPModel(nn.Module):
    def __init__(self, fc1,fc2,relu):
        super().__init__()
        self.fc1 = fc1
        self.fc2 = fc2
        self.relu=relu

    def forward(self, x):
        if x.dim() == 2:  # (features, time)
            x = x.unsqueeze(0)

        with torch.no_grad():
            out = self.fc1(x)
            out = self.relu(out)
            out = self.fc2(out)
        return out






In [9]:
import torch
import io
import torch.nn.functional as F

# ----- Load the .pt file -----
pt_path = "/srv/DEEP-OC-underwater-noise-classification/data/G_15810_2022-02-07_08-21-29_156-760693_Cargo_underway-using-engine_12-3_1768.pt"
with open(pt_path, "rb") as f:
    file_bytes = f.read()

file_like = io.BytesIO(file_bytes)
embedding = torch.load(file_like, map_location="cpu")

print(f"Loaded object type: {type(embedding)}")
print(f"Shape: {getattr(embedding, 'shape', 'No shape')}")

# ----- Prepare the model -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model_loader(device, freeze=True)
model.eval()

# ----- Run prediction -----
with torch.no_grad():
    if isinstance(embedding, torch.Tensor):
        print(len(embedding))
        embedding = embedding.to(device)
        output = model(embedding)[0]
        print(output[0])
        predicted_class = torch.argmax(output, dim=1).item()
        probabilities = F.softmax(output, dim=1).squeeze().cpu().tolist()

        print(f"✅ Predicted class: {predicted_class}")
        print(f"✅ Class probabilities: {probabilities}")
    else:
        print("❌ Loaded object is not a tensor")

Loaded object type: <class 'torch.Tensor'>
Shape: torch.Size([1, 512])
1
tensor([ 0.1399,  0.1999,  0.2344,  0.2434,  0.2392,  0.2089,  0.1574,  0.0785,
        -0.0233, -0.1908, -0.4025])
✅ Predicted class: 3
✅ Class probabilities: [0.09475743025541306, 0.10062021017074585, 0.104147769510746, 0.10509079694747925, 0.1046474352478981, 0.10153113305568695, 0.09643180668354034, 0.08911938220262527, 0.08048717677593231, 0.06807927042245865, 0.05508754029870033]


In [11]:
import torch
import io
import torch.nn.functional as F

# ----- Load the .pt file -----
pt_path = "/srv/DEEP-OC-underwater-noise-classification/data/G_15810_2022-02-07_08-21-29_156-760693_Cargo_underway-using-engine_12-3_1768 (1).pt"
with open(pt_path, "rb") as f:
    file_bytes = f.read()

file_like = io.BytesIO(file_bytes)
embedding = torch.load(file_like, map_location="cpu")

print(f"Loaded object type: {type(embedding)}")
print(f"Shape: {getattr(embedding, 'shape', 'No shape')}")

# ----- Prepare the model -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model_loader(device, freeze=False)
model.eval()

# ----- Run prediction -----
with torch.no_grad():
    if isinstance(embedding, torch.Tensor):
        embedding = embedding.to(device)
        print(embedding)
        output = model(embedding)
        print(output)
        predicted_class = torch.argmax(output, dim=1).item()
        probabilities = F.softmax(output, dim=1).squeeze().cpu().tolist()

        print(f"✅ Predicted class: {predicted_class}")
        print(f"✅ Class probabilities: {probabilities}")
    else:
        print("❌ Loaded object is not a tensor")

Loaded object type: <class 'torch.Tensor'>
Shape: torch.Size([1, 1, 1001, 64])
tensor([[[[-16.4730, -32.7780, -26.3730,  ..., -62.5544, -61.8940, -62.5178],
          [-15.5435, -30.7138, -33.1804,  ..., -62.8598, -61.7451, -65.3546],
          [ -6.8622, -17.5188, -31.9298,  ..., -61.4319, -61.5811, -62.9530],
          ...,
          [ -9.4487, -25.2350, -25.3325,  ..., -60.9797, -64.2304, -62.7755],
          [-14.5285, -22.4340, -22.2966,  ..., -60.7422, -61.3189, -63.8114],
          [-11.2158,  -8.8075, -13.1319,  ..., -62.9723, -60.0463, -65.6816]]]])
tensor([[ 0.1266,  0.4500,  0.6062,  0.6953,  0.6020,  0.3639,  0.0129, -0.4810,
         -0.9104, -1.1478, -1.4659]])
✅ Predicted class: 3
✅ Class probabilities: [0.09162174165248871, 0.12660335004329681, 0.1480112224817276, 0.16180580854415894, 0.14738497138023376, 0.116162970662117, 0.08177360892295837, 0.049903158098459244, 0.03248002752661705, 0.025616005063056946, 0.018637174740433693]


In [43]:
output

tensor([[[ 0.1399,  0.1999,  0.2344,  0.2434,  0.2392,  0.2089,  0.1574,
           0.0785, -0.0233, -0.1908, -0.4025]]])

In [34]:
import torch
import io
import torch.nn.functional as F

# ----- Load the .pt file -----
pt_path = "/srv/DEEP-OC-underwater-noise-classification/data/G_15810_2022-02-07_08-21-29_156-760693_Cargo_underway-using-engine_12-3_1768 (1).pt"
with open(pt_path, "rb") as f:
    file_bytes = f.read()

file_like = io.BytesIO(file_bytes)
embedding = torch.load(file_like, map_location="cpu")

print(f"Loaded object type: {type(embedding)}")
print(f"Shape: {getattr(embedding, 'shape', 'No shape')}")

# ----- Prepare the model -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model_loader(device)
model.eval()

# ----- Run prediction -----
with torch.no_grad():
    if isinstance(embedding, torch.Tensor):
        embedding = embedding.to(device)
        output = model(embedding)

        predicted_class = torch.argmax(output, dim=1).item()
        probabilities = F.softmax(output, dim=1).squeeze().cpu().tolist()

        print(f"✅ Predicted class: {predicted_class}")
        print(f"✅ Class probabilities: {probabilities}")
    else:
        print("❌ Loaded object is not a tensor")


Loaded object type: <class 'torch.Tensor'>
Shape: torch.Size([1, 1, 1001, 64])
✅ Predicted class: 3
✅ Class probabilities: [0.09162174165248871, 0.12660335004329681, 0.1480112224817276, 0.16180580854415894, 0.14738497138023376, 0.116162970662117, 0.08177360892295837, 0.049903158098459244, 0.03248002752661705, 0.025616005063056946, 0.018637174740433693]


In [30]:
model = model_loader()
logits = model(embedding.to(device))
predicted_class = torch.argmax(logits, dim=1).item()
probabilities = torch.softmax(logits, dim=1).squeeze().cpu().tolist()


NameError: name 'model_loader' is not defined

In [27]:
predicted_class

3

In [26]:
probabilities

[0.09162174165248871,
 0.12660335004329681,
 0.1480112224817276,
 0.16180580854415894,
 0.14738497138023376,
 0.116162970662117,
 0.08177360892295837,
 0.049903158098459244,
 0.03248002752661705,
 0.025616005063056946,
 0.018637174740433693]

In [24]:
len(clap_output.audio_embeds[0])

512

In [21]:
clap_output.audio_embeds

tensor([[ 2.8619e-02, -1.3301e-01,  2.0999e-01, -2.2199e-01,  2.1193e-02,
         -3.1606e-01,  2.1937e-01, -2.1222e-01, -2.4672e-01,  4.3776e-03,
         -3.0062e-01,  1.5420e-01,  1.5935e-01,  5.8744e-02, -6.6256e-03,
         -3.9827e-01, -2.8521e-01, -1.5809e-01, -3.1395e-01,  4.2836e-01,
          1.2994e-01, -1.1704e-01, -4.5699e-01,  4.1617e-02,  6.7006e-02,
         -3.5563e-02, -5.7896e-02, -4.2090e-03, -1.6218e-01, -3.9658e-02,
          4.0642e-01,  1.6578e-01, -3.1217e-01,  7.3475e-03, -2.7456e-01,
         -1.2519e-01,  2.4036e-01, -3.5438e-01, -1.8330e-01,  1.8214e-01,
         -4.9875e-02, -1.0472e-01, -1.1271e-02,  1.8519e-02,  3.6128e-01,
         -8.7712e-02,  1.5493e-01, -1.2605e-01, -3.3798e-02,  2.4694e-01,
         -7.2173e-02, -3.5188e-02, -2.7408e-01,  2.0994e-02, -4.3836e-02,
          4.5556e-02, -6.5583e-02,  4.4070e-02, -1.3603e-02, -2.4973e-01,
         -2.9477e-01, -2.4779e-01, -1.1213e-01, -1.0888e-01,  7.8344e-02,
         -1.8834e-02,  2.6447e-01, -1.